## Vektoren
* Sind für unsere Zwecke "Listen" oder "Tupel" von Zahlen.
Zum beispiel [1, 3, 2, 10], oder [0, 0, 1] die man horizontal oder vertikal schreiben kann.
* Vektoren (mit gleicher Anzahl Elementen!) kann man zusammenzählen und mit einem Skalar multiplizieren.
* $$ \left(\begin{matrix}2 \\ 5 \\ 1\end{matrix}\right)  + \left(\begin{matrix}1 \\ -1 \\ 0\end{matrix}\right) = \left(\begin{matrix}3 \\ 4 \\ 1\end{matrix}\right)$$


* $$ 3 \; \left(\begin{matrix}2 \\ 5 \\ 1\end{matrix}\right) = \left(\begin{matrix}6 \\ 15 \\ 3\end{matrix}\right)$$

## Skalarprodukt
* Wird in der Mathematik mit einem Punkt geschrieben. Für zwei Vektoren $a$ und $b$ also $a \cdot b$


* Ist ein Mass für den Winkel zwischen den zwei Vektoren.
* Falls Skalarprodukt > 0 --> Vektoren zeigen in dieselbe Richtung
* Falls Skalarprodukt = 0 --> Vektoren senkrecht zueinander
* Falls Skalarprodukt < 0 --> Vektoren zeigen in entgegengesetzte Richtungen

* In Julia ist das Skalarprodukt eine eingebaute Funktion und heist "dot()". Also dot(a, b) entspricht dem $ a \cdot b$

* $a_1 b_1 + a_2b_2 + \dots + a_nb_n$

In [1]:
# Import the necessary libraries to work with tabulated
# data in Julia and show results
using DataFrames
using StatPlots
using Plots
using Optim

In [68]:
# Lade Daten und speichere sie in der Variable "df" ab.
# df steht hier für DataFrame und ist vergleichbar mit
# einer Excel Tabelle.
# Jede Spalte der Tabelle ist dabei eine Variable.
# Jede Zeile der Tabelle ist eine "Messung"
# Geschlecht ist die Zielvariable, das was wir voraussagen wollen.
# 1 := maennlich, 0 := weiblich

df = readtable("data_ex2.csv")
#raw_data  = convert(Matrix{Float64}, df[:, [:UmfangBrust, :UmfangHuefte, :UmfangBizeps, :UmfangHandgelenk, :Alter, :Gewicht,:Groesse]])
raw_data  = convert(Matrix{Float64}, df[:, [:UmfangHandgelenk, :Alter]])

X_norm = (raw_data .- mean(raw_data, 1)) ./ std(raw_data, 1)

const X = hcat(ones(size(X_norm, 1)), X_norm)
const y = convert(Vector{Float64}, df[:,:Geschlecht]);

In [56]:
@show(size(df))
head(df)

size(df) = (507,8)


,UmfangBrust,UmfangHuefte,UmfangBizeps,UmfangHandgelenk,Alter,Gewicht,Groesse,Geschlecht
1,89.5,93.5,32.5,16.5,21,65.6,174.0,1
2,97.0,94.8,34.4,17.0,23,71.8,175.3,1
3,97.5,95.0,33.4,16.9,28,80.7,193.5,1
4,97.0,94.0,31.0,16.6,23,72.6,186.5,1
5,97.5,98.5,32.0,18.0,22,78.8,187.2,1
6,99.9,95.3,33.0,16.9,21,74.8,181.5,1


In [49]:
# Darstellen der Messpunkte
scatter(df, :UmfangHuefte, :Groesse, group=:Geschlecht)

# Wähle zwei Variablen aus die du denkst sind vielversprechend

# Hypothese
Unser Ziel ist es, an hand der Messungen eine Vorhersage über das Geschlecht
der Person machen zu können. Das heisst für jede Messung wollen wir berechnen ob die Zielvariable = 1 (Mann) oder 0 (Frau) ist.

Man stellt mit Hilfe des Skalarprodukts eine Hypothese "H" auf um die Zielvariable y vorauszusagen.
H = dot(x, theta)

Unsere Hypothese ist also:

$$Geschlecht = y \approx H(s)$$

$$s = x_1\theta_1 + x_2 \theta_2 + \cdots + x_n \theta_n$$

Mit den unbekannten $\theta_i$ und den Messungen/Variablen $x_i$:

* $x_1$ = :UmfangBrust
* $x_2$ = :UmfangHuefte
* $\vdots$
* $x_7$ = :Groesse
* y = :Geschlecht


$s$ ist das Skalarprodukt zwischen $x$ und $\theta$. Zur Erinnerung:
$s$ kann eine Zahl zwischen -Unendlich bis +Unendlich sein.


Weil die Zielvariable nur entweder 0 oder 1 sein kann, brauchen wir also noch eine Funktion, die das Skalarprodukt nimmt und dann 0 oder 1 ausspuckt. Diese Funktion ist $H(s)$ (sprich: Ha von Ess)


Wir werden als erstes die folgende Funktion verwenden:

$H(s)$ = 1 falls s $\geq$ 0

$H(s)$ = 0 falls s $<$ 0

# Lösungsvorgehen
Nun müssen $\theta_1$ bis $\theta_7$ so gefunden werden, dass unsere
Hypothese möglichst gut funktioniert. Das ist Machine Learning.

# Optimierungsprobleme
Das finden von $\theta$ für unsere Aufgabe ist ein Optimierungsproblem. **Finde $\theta$ so dass $H(s)$ möglichst gut funktioniert!**. Es gitb sehr viele verschiedene Methoden um dieses Optimum zu finden und in der Regel können wir einfach eine dieser fertigen Methoden benutzen.

Als erstes brauchen wir also nur noch eine definition was überhaupt "gut" ist. Am liebsten wird diese Qualität als Skalar gemessen. Das Ziel ist dann diese Zahl möglichst klein/gross zu machen. Z.b.

* **Gewinn** einer Firma (mehr ist besser).
* **zu fahrende Kilometer** für einen Amazon Lieferwagen (weniger ist besser)


Julia hat ein Modul **Optim** welche verschiedene optimierungsverfahren bereitstellt.
Um damit Probleme zu lösen müssen wir eine Funktion definieren die ein $\theta$ annimmt,
und dann eine Zahl zurück gibt welche die Qualität von $\theta$ beschreibt.


1) bestimme einen Anfangswert für $\theta$.

2) Aus $\theta$ und $X$ bekommen wir das Skalarprodukt $s$.

3) H(s) ergibt unsere Vorhersage für "Geschlecht".

4) Vergleiche Vorhersage und "Geschlecht" um Qualität zu bestimmen.

5) Ändere $\theta$ so ab, dass Qualität zunimmt* --> (das macht Optim)

In [79]:
# Definition der Funktionen für optimize()
function H(s)
    if s >= 0
        return 1
    else
        return 0
    end
end

function cost(theta)
    s = X * theta
    prediction = H.(s)
    accuracy = mean(prediction .== y)
    score = -accuracy
    return score
end

cost (generic function with 1 method)

In [85]:
# Finde einen Anfangswert für theta
theta_init = randn(size(X, 2))

# Starte Optimierung
opt = optimize(cost, theta_init)

# Zeige Resultate
theta_opt = opt.minimizer
accuracy = opt.minimum
@show(accuracy)
@show(theta_opt);

accuracy = -0.8954635108481263
theta_opt = [0.195846,3.45667,0.425519]
